<a href="https://colab.research.google.com/github/nianlonggu/MemSum/blob/main/Training_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
!git clone https://github.com/nianlonggu/MemSum.git

Cloning into 'MemSum'...
remote: Enumerating objects: 363, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 363 (delta 40), reused 21 (delta 6), pack-reused 266
Receiving objects: 100% (363/363), 82.37 MiB | 14.20 MiB/s, done.
Resolving deltas: 100% (137/137), done.


change work directory to MemSum

In [ ]:
import os
os.chdir("MemSum")

## Set Up Environment

Install dependencies via pip

In [ ]:
!pip install -r requirements.txt --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 47.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:0

# Preprocessing Custom data

Suppose that you have already splitted the training / validation and  test set:

The training data is now stored in a .jsonl file that contains a list of json info, one line for one training instance. Each json (or dictonary) contains two keys:

1. "text": the value for which is a python list of sentences, this represents the document you want to summarize;
2. "summary": the value is also a list of sentences. If represent the ground-truth summary. Because the summary can contain multiple sentences, so we store them as a list.

The same for the validation file and the testing file.


In [ ]:
import json
train_corpus = [ json.loads(line) for line in open("data/custom_data/train_raw_without_high_rouge_indices_and_scores.jsonl") ]

## as an example, we have 100 instances for training
print(len(train_corpus))
print(train_corpus[0].keys())
print(train_corpus[0]["text"][:3])
print(train_corpus[0]["summary"][:3])

100
dict_keys(['text', 'summary'])
['a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .', 'in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .', 'the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .']
['background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz 

If you have your own data, process them into the same structure then put them into the data/ folder

The next thing we need to do is to create high-ROUGE episodes for the training set, as introduced in the paper: https://aclanthology.org/2022.acl-long.450/:

Note:
* The '!' in the following commands is only needed when running shell command in a jupyter notebook
* Thie script run multi-process tasks. You can change -n_processes to a large value on a machine with powerful multi-core CPUs.

In [ ]:
!cd data_preprocessing; python get_high_rouge_episodes_mp.py -input_corpus_file_name ../data/custom_data/train_raw_without_high_rouge_indices_and_scores.jsonl -output_corpus_file_name ../data/custom_data/train.jsonl -beamsearch_size 2 -n_processes 10


70it [01:28,  1.26s/it]
finished!
30it [01:35,  3.18s/it]
finished!
90it [01:36,  1.08s/it]
finished!
50it [01:36,  1.93s/it]
finished!
10it [01:39,  9.99s/it]
finished!
20it [01:44,  5.21s/it]
finished!
40it [01:44,  2.62s/it]
finished!
80it [01:45,  1.32s/it]
finished!
60it [01:45,  1.75s/it]
finished!
100it [01:46,  1.06s/it]
finished!


# Download pretrained word embedding

In [ ]:
from huggingface_hub import snapshot_download
## download the pretrained glove word embedding (200 dimension)
snapshot_download('nianlong/memsum-word-embedding', local_dir = "model/word_embedding" )

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

'/content/MemSum/model/word_embedding'

# Start Training

Note:
1. you need to switch to the folder src/MemSum_Full;
2. You can specify the path to training and validation set, the model_folder (where you want to store model checkpoints) and the log_folder (where you want to store the log info), and other parameters.
3. You can provide the absolute path, or relative path, as shown in the example code below.
4. n_device means the number of available GPUs
5. set save_every and validate_every to 0 will make the training script save and evaluate the model only at the end of each epoch.

In [ ]:
!cd src; python train.py -training_corpus_file_name ../data/custom_data/train.jsonl -validation_corpus_file_name ../data/custom_data/val.jsonl -model_folder ../model/memsum-custom-data -log_folder ../log/memsum-custom-data -vocabulary_file_name ../model/word_embedding/vocabulary_200dim.pkl -pretrained_unigram_embeddings_file_name ../model/word_embedding/unigram_embeddings_200dim.pkl -max_seq_len 100 -max_doc_len 500 -num_of_epochs 10 -save_every 0 -validate_every 0 -n_device 1 -batch_size_per_device 4 -max_extracted_sentences_per_document 7 -moving_average_decay 0.999 -p_stop_thres 0.6


100it [00:00, 12944.58it/s]
100it [00:00, 17219.41it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
24it [00:26,  1.04s/it]Starting validation ...
[current_batch: 00025] val: 0.3182, 0.0907, 0.2875
25it [00:39,  1.59s/it]
24it [00:26,  1.09s/it]Starting validation ...
[current_batch: 00050] val: 0.3201, 0.0906, 0.2884
25it [00:40,  1.63s/it]
24it [00:27,  1.13s/it]Starting validation ...
[current_batch: 00075] val: 0.3233, 0.0923, 0.2899
25it [00:41,  1.68s/it]
24it [00:26,  1.12s/it][current_batch: 00100] loss: 0.509, learning rate: 0.000100
Starting validation 

# Determine the best checkpoint based on validation loss

In [ ]:
! cd src; python get_optimal_batch.py -log_file_name ../log/memsum-custom-data/val.log

batch: 225 scores: ('0.3721', '0.1236', '0.3327')


So the best checkpoint is model_batch_200.pt

# Testing trained model on custom dataset

In [ ]:
from src.summarizer import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np

In [ ]:
rouge_cal = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeLsum'], use_stemmer=True)

memsum_custom_data = MemSum(  "model/memsum-custom-data/model_batch_200.pt",
                  "model/word_embedding/vocabulary_200dim.pkl",
                  gpu = 0 ,  max_doc_len = 500  )

In [ ]:
test_corpus_custom_data = [ json.loads(line) for line in open("data/custom_data/test.jsonl")]

In [ ]:
def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]

        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )

    return np.asarray(scores).mean(axis = 0)

In [ ]:
evaluate( memsum_custom_data, test_corpus_custom_data, 0.6, 7, rouge_cal )

100%|██████████| 100/100 [00:16<00:00,  6.01it/s]


array([0.37647216, 0.12130708, 0.335058  ])

In [ ]:
document = test_corpus_custom_data[0]["text"]
extracted_summary = memsum_custom_data.extract( [ document ],
                                   p_stop_thres = 0.6,
                                   max_extracted_sentences_per_document = 7
                                  )[0]
extracted_summary

['ten patients in this study ( 5 pd with anxiety ; 5 pd without anxiety ) were taking psychotropic drugs ( i.e. , benzodiazepine or selective serotonin reuptake inhibitor ) .',
 "anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] .",
 'selective serotonin reuptake inhibitor , citalopram , was shown to improve response inhibition deficits in pd , while noradrenaline reuptake blocker , atomoxetine , has been recently reported to have promising effects on cognition in pd [ 45 , 46 ] .',
 'overall , neuropsychiatric symptoms in pd have been shown to be negatively associated with cognitive performance .',
 'although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet ,',
 'our current understanding of anxiety and its impact on